In [2]:
# ===========================
# SmartBiz Dashboard
# Author: Sudhama Mattegunta
# Purpose: Customer Analytics, Transactions Risk, Inventory Trends, Feedback Sentiment
# Features: Interactive filters using ipywidgets, SQLite3 support,  visualizations
# ===========================

# 0️⃣ Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from ipywidgets import interact, widgets
from IPython.display import display
import sqlite3
%matplotlib inline

sns.set(style="whitegrid")

# ===========================
# 1️⃣ Database Setup
# ===========================
conn = sqlite3.connect('smartbiz.db')
cursor = conn.cursor()

# Customers table
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers(
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    last_purchase_date TEXT,
    total_spent REAL,
    total_orders INTEGER
)
''')
cursor.execute('SELECT COUNT(*) FROM customers')
if cursor.fetchone()[0] == 0:
    sample_customers = [
        (1,'Alice','2025-03-15',1200,5),
        (2,'Bob','2025-08-01',450,2),
        (3,'Charlie','2024-12-20',2200,10),
        (4,'David','2025-01-05',800,3)
    ]
    cursor.executemany('INSERT INTO customers VALUES (?,?,?,?,?)', sample_customers)
    conn.commit()

# Transactions table
cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions(
    transaction_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    amount REAL,
    date TEXT
)
''')
cursor.execute('SELECT COUNT(*) FROM transactions')
if cursor.fetchone()[0] == 0:
    sample_transactions = [
        (1,1,500,'2025-03-16'),
        (2,2,450,'2025-08-02'),
        (3,3,60000,'2025-01-10'),
        (4,4,1500,'2025-01-06')
    ]
    cursor.executemany('INSERT INTO transactions VALUES (?,?,?,?)', sample_transactions)
    conn.commit()

# Inventory table
cursor.execute('''
CREATE TABLE IF NOT EXISTS inventory(
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    stock_qty INTEGER,
    weekly_sales INTEGER
)
''')
cursor.execute('SELECT COUNT(*) FROM inventory')
if cursor.fetchone()[0]==0:
    sample_inventory = [
        (1,'Product A',100,20),
        (2,'Product B',50,15),
        (3,'Product C',200,40)
    ]
    cursor.executemany('INSERT INTO inventory VALUES (?,?,?,?)', sample_inventory)
    conn.commit()

# Feedback table
cursor.execute('''
CREATE TABLE IF NOT EXISTS feedback(
    feedback_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    review_text TEXT,
    rating INTEGER
)
''')
cursor.execute('SELECT COUNT(*) FROM feedback')
if cursor.fetchone()[0]==0:
    sample_feedback = [
        (1,1,'Great service!',5),
        (2,2,'Bad experience',2),
        (3,3,'Very satisfied',4),
        (4,4,'Not happy',1)
    ]
    cursor.executemany('INSERT INTO feedback VALUES (?,?,?,?)', sample_feedback)
    conn.commit()

conn.close()

# ===========================
# 2️⃣ Load Data Function
# ===========================
def load_data():
    conn = sqlite3.connect('smartbiz.db')
    customers = pd.read_sql_query('SELECT * FROM customers', conn)
    transactions = pd.read_sql_query('SELECT * FROM transactions', conn)
    inventory = pd.read_sql_query('SELECT * FROM inventory', conn)
    feedback = pd.read_sql_query('SELECT * FROM feedback', conn)
    conn.close()

    # Inventory moving avg
    if 'weekly_sales' not in inventory.columns:
        inventory['weekly_sales'] = [0]*len(inventory)
    inventory['Moving_Avg'] = inventory['weekly_sales'].rolling(window=3,min_periods=1).mean()

    # Feedback sentiment
    feedback['Sentiment'] = feedback['review_text'].apply(lambda x: 'Positive' if TextBlob(x).sentiment.polarity>0 else 'Negative')

    # Customer analytics
    today = pd.to_datetime('2025-09-13')
    customers['Recency'] = (today - pd.to_datetime(customers['last_purchase_date'])).dt.days
    customers['Segment'] = pd.cut(customers['total_spent'], bins=[0,500,1500,10000], labels=['Low','Medium','High'])
    customers['Churn_Risk'] = customers['Recency'].apply(lambda x: 'High' if x>180 else 'Low')

    # Transaction risk
    transactions['Risk_Flag'] = transactions['amount'].apply(lambda x: 'High' if x>50000 else 'Normal')

    return customers, transactions, inventory, feedback

# ===========================
# 3️⃣ Dashboard Function
# ===========================
def show_dashboard(segment):
    customers, transactions, inventory, feedback = load_data()

    # Segment filter
    cust_df = customers if segment=='All' else customers[customers['Segment']==segment]

    print("=== Customers ===")
    display(cust_df)

    print("=== High-Risk Transactions ===")
    display(transactions[transactions['Risk_Flag']=='High'])

    # Inventory plot
    plt.figure(figsize=(8,4))
    sns.lineplot(data=inventory, x='product_name', y='Moving_Avg', marker='o', color='navy')
    plt.title("Inventory Moving Average per Product", fontsize=14)
    plt.xlabel("Product")
    plt.ylabel("Moving Avg Sales")
    plt.show()

    # Feedback sentiment
    plt.figure(figsize=(6,4))
    sns.countplot(x='Sentiment', data=feedback, hue='Sentiment', dodge=False, palette=['lightgreen','salmon'])
    plt.title("Customer Feedback Sentiment", fontsize=14)
    plt.legend([],[], frameon=False)
    plt.show()

    # Interactive filters
    def show_customers_widget(seg):
        display(customers[customers['Segment']==seg])
    interact(show_customers_widget, seg=['Low','Medium','High'])

    def churn_pie(days):
        temp = customers.copy()
        temp['Churn_Risk'] = temp['Recency'].apply(lambda x: 'High' if x>days else 'Low')
        temp['Churn_Risk'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['lightcoral','lightgreen'])
        plt.title(f"Churn Risk > {days} Days", fontsize=14)
        plt.ylabel("")
        plt.show()
    interact(churn_pie, days=(30,365,30))

    def inventory_trend(product_name):
        temp = inventory[inventory['product_name']==product_name]
        sns.lineplot(data=temp, x='product_name', y='Moving_Avg', marker='o', color='blue')
        plt.title(f"{product_name} – Inventory Trend", fontsize=14)
        plt.xlabel("Product")
        plt.ylabel("Moving Avg")
        plt.show()
    interact(inventory_trend, product_name=inventory['product_name'].tolist())

    def show_risk(threshold):
        temp = transactions[transactions['amount']>threshold]
        print(f"Transactions above {threshold}:")
        display(temp)
    interact(show_risk, threshold=(1000,100000,5000))

    def show_feedback_widget(sentiment):
        display(feedback[feedback['Sentiment']==sentiment])
    interact(show_feedback_widget, sentiment=['Positive','Negative'])

# ===========================
# 4️⃣ Link Widget
# ===========================
widgets.interactive(show_dashboard, segment=widgets.Dropdown(options=['All','Low','Medium','High'], description='Segment:'))


interactive(children=(Dropdown(description='Segment:', options=('All', 'Low', 'Medium', 'High'), value='All'),…